In [21]:
from kafka import  KafkaProducer
import sys, os, json
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import os
import sys
from pyspark.sql.functions import *
from pyspark.sql.functions import to_json, struct
import kafka
from pyspark.sql.functions import to_json, col, struct

In [2]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print( ' Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))


def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_server=['localhost:9092'], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka', ex)
        print(str(ex))
    finally:
        return _producer

In [3]:
topic_name = 'quickstart-events4'
bootstrap_server="localhost:9092"

In [4]:
scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = '3.3.0'

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.5.1'
]

args = os.environ.get('PYSPARK_SUBMIT_ARGS', '')
if not args:
    args = f'--packages {",".join(packages)}'
    print('Using packages', packages)
    os.environ['PYSPARK_SUBMIT_ARGS'] = f'{args} pyspark-shell'
else:
    print(f'Found existing args: {args}')

Using packages ['org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0', 'org.apache.kafka:kafka-clients:3.5.1']


In [5]:
# set Java home
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk1.8.0_251"

In [6]:
conf = SparkConf() \
    .setAppName("ETLPipeline") \
    .setMaster("local") \
    .set("spark.driver.extraClassPath","c:/pyspark/*")
sc = SparkContext.getOrCreate(conf=conf)
etl = SparkSession(sc)

In [7]:
uid="chaithanya"
pwd="12345678"

#sql db details
server = "localhost"
src_db = "prac"
src_driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

In [9]:
src_url = f"jdbc:sqlserver://{server}:1433;databaseName={src_db};user={uid};password={pwd};"

In [10]:
query="""select emp_id,emp_name,salary,manager_id from prac.dbo.emp_manager"""
table="emp_manager"

In [11]:
dfs=etl.read. \
    format("jdbc"). \
    options(driver=src_driver, user=uid, password=pwd, url=src_url,query=query). \
    load()
dfs.show()

+------+--------+------+----------+
|emp_id|emp_name|salary|manager_id|
+------+--------+------+----------+
|     1|   Ankit| 10000|         4|
|     2|   Mohit| 15000|         5|
|     3|   Vikas| 10000|         4|
|     4|   Rohit|  5000|         2|
|     5|   Mudit| 12000|         6|
|     6|    Agam| 12000|         2|
|     7|  Sanjay|  9000|         2|
|     8|  Ashish|  5000|         2|
+------+--------+------+----------+



In [20]:
kafka_df = dfs.selectExpr("CAST(emp_id AS STRING) AS key", "to_json(struct(*)) AS value")
from datetime import datetime, timedelta
#pd_df = dfs.toPandas()
#  Reads data from sql file and published those  data as json format to the topic 

kafka_producer = connect_kafka_producer()

#  iterate over the dataframe and send rows as the value, and keys can be the desired column
for index, row in kafka_df.iterrows(): 
    publish_message(kafka_producer, topic_name, "key",  "value") 
    
import pandas as pd
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
pandasDF = df.toPandas()
for index, row in pandasDF.iterrows():   
    publish_message(kafka_producer, topic_name, "key",  "value") 
    
if kafka_producer is not None:
    kafka_producer.close()

Exception while connecting Kafka Unrecognized configs: {'bootstrap_server': ['localhost:9092']}
Unrecognized configs: {'bootstrap_server': ['localhost:9092']}


AttributeError: 'DataFrame' object has no attribute 'iterrows'